# Creare un dashboard di intelligenza artificiale responsabile per valutare i modelli

Quando si confrontano e si valutano i modelli di Machine Learning, è consigliabile esaminare più semplicemente la metrica delle prestazioni. Azure Machine Learning consente di creare un dashboard di intelligenza artificiale responsabile per esplorare le prestazioni del modello in coorti diverse dei dati. 

## Preparare i dati

Per creare il dashboard di intelligenza artificiale responsabile, è necessario un set di dati di training e test, archiviato come file Parquet e registrato come asset di dati.

I dati vengono attualmente archiviati come file CSV. Convertirli in file Parquet. 


In [ ]:
## Prima di continuare

Per eseguire il codice in questo notebook, è necessaria la versione più recente del pacchetto **azureml-ai-ml** . Eseguire la cella seguente per verificare che sia installata.

> **Nota**:
> Se il pacchetto **azure-ai-ml** non è installato, eseguire `pip install azure-ai-ml` per installarlo.

In [ ]:
## Connettersi all'area di lavoro

Dopo aver installato i pacchetti SDK necessari, è ora possibile connettersi all'area di lavoro.

Per connettersi a un'area di lavoro, sono necessari parametri di identificatore: ID sottoscrizione, nome del gruppo di risorse e nome dell'area di lavoro. Il nome del gruppo di risorse e il nome dell'area di lavoro sono già compilati automaticamente. Per completare il comando è necessario solo l'ID sottoscrizione.

Per trovare i parametri necessari, fare clic sulla sottoscrizione e sul nome dell'area di lavoro in alto a destra di Studio. Verrà aperto un riquadro a destra.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiare l'ID sottoscrizione e sostituire **YOUR-SUBSCRIPTION-ID** con il valore copiato. </p>

## Creare gli asset di dati

Per creare il dashboard di intelligenza artificiale responsabile, è necessario registrare i set di dati di training e test come asset di dati **MLtable** . Gli asset di dati MLtable fanno riferimento ai file Parquet creati in precedenza.

In [ ]:
## Creare la pipeline per creare il dashboard di intelligenza artificiale responsabile

Per creare il dashboard, si creerà una pipeline con componenti predefiniti registrati per impostazione predefinita nell'area di lavoro di Azure Machine Learning.

### Registrare il modello

È già stato eseguito il training di un modello di Machine Learning. Il modello stima se un paziente ha il diabete. Tutti i file del modello vengono archiviati nella `model` cartella . 

Registrare il modello puntando alla `model` cartella e al relativo contenuto.

In [ ]:
## Creare la pipeline

Per creare il dashboard di intelligenza artificiale responsabile, si creerà una pipeline usando i componenti predefiniti. È possibile scegliere quali componenti usare e quali funzionalità includere nel dashboard. Verrà creato un dashboard che include l'analisi degli errori e l'interpretazione del modello. 

Accanto alle funzionalità di intelligenza artificiale responsabili, una pipeline per creare un dashboard deve includere un componente all'inizio per costruire il dashboard e un componente alla fine per raccogliere tutte le informazioni dettagliate generate.

In [ ]:
Dopo aver recuperato tutti i componenti da usare, è possibile compilare la pipeline e connettere i componenti nell'ordine appropriato:

1. Costruire il dashboard.
1. Aggiungere l'analisi degli errori.
1. Aggiungere spiegazioni.
1. Raccogliere tutte le informazioni dettagliate e visualizzarle nel dashboard.

Ora la pipeline è stata compilata, è necessario definire i due input necessari: il set di dati di training e di test.

In [ ]:
Infine, verranno inseriti tutti gli elementi: assegnare gli input alla pipeline e impostare la colonna di destinazione (etichetta stimata).

In [ ]:
## Eseguire la pipeline

Dopo aver compilato correttamente la pipeline, è possibile inviarla. Il codice seguente invia la pipeline e controlla lo stato della pipeline. È anche possibile visualizzare lo stato della pipeline in Studio. 

Al termine della pipeline, è possibile esaminare il dashboard in Studio. 

In [ ]:
# Get handle to azureml registry for the RAI built in components
registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    registry_name=registry_name,
)
print(ml_client_registry)

### Register the model

A machine learning model has already been trained for you. The model predicts whether a patient has diabetes. All model files are stored in the `model` folder. 

Register the model by pointing to the `model` folder and its contents.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="model",
    type=AssetTypes.MLFLOW_MODEL,
    name="local-mlflow-diabetes",
    description="Model created from local file.",
)
model = ml_client.models.create_or_update(file_model)

## Build the pipeline

To create the responsible AI dashboard, you'll create a pipeline using the prebuilt components. You can choose which components to use, and which features you want to include in your dashboard. You'll create a dashboard that includes error analysis and model interpretability. 

Next to the responsible AI features, a pipeline to build a dashboard needs to include a component at the start to construct the dashboard, and a component at the end to gather all generated insights.

In [ ]:
model_name = model.name
expected_model_id = f"{model_name}:1"
azureml_model_id = f"azureml:{expected_model_id}"

In [ ]:
label = "latest"

rai_constructor_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_constructor", label=label
)

# we get latest version and use the same version for all components
version = rai_constructor_component.version
print("The current version of RAI built-in components is: " + version)

rai_erroranalysis_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_erroranalysis", version=version
)

rai_explanation_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_explanation", version=version
)

rai_gather_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_gather", version=version
)

When you've retrieved all components you want to use, you can build the pipeline and connect the components in the appropriate order:

1. Construct the dashboard.
1. Add error analysis.
1. Add explanations.
1. Gather all insights and visualize them in the dashboard.

In [ ]:
from azure.ai.ml import Input, dsl
from azure.ai.ml.constants import AssetTypes

compute_name = "aml-cluster"

@dsl.pipeline(
    compute=compute_name,
    description="RAI insights on diabetes data",
    experiment_name=f"RAI_insights_{model_name}",
)
def rai_decision_pipeline(
    target_column_name, train_data, test_data
):
    # Initiate the RAIInsights
    create_rai_job = rai_constructor_component(
        title="RAI dashboard diabetes",
        task_type="classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        train_dataset=train_data,
        test_dataset=test_data,
        target_column_name=target_column_name,
    )
    create_rai_job.set_limits(timeout=30)

    # Add error analysis
    error_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
    )
    error_job.set_limits(timeout=10)

    # Add explanations
    explanation_job = rai_explanation_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        comment="add explanation", 
    )
    explanation_job.set_limits(timeout=10)

    # Combine everything
    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,
        insight_3=error_job.outputs.error_analysis,
        insight_4=explanation_job.outputs.explanation,
    )
    rai_gather_job.set_limits(timeout=10)

    rai_gather_job.outputs.dashboard.mode = "upload"

    return {
        "dashboard": rai_gather_job.outputs.dashboard,
    }

Now the pipeline has been built, you need to define the two necessary inputs: the training and test dataset.

In [ ]:
from azure.ai.ml import Input
target_feature = "Diabetic"

diabetes_train_pq = Input(
    type="mltable",
    path=f"azureml:{input_train_data}:{data_version}",
    mode="download",
)
diabetes_test_pq = Input(
    type="mltable",
    path=f"azureml:{input_test_data}:{data_version}",
    mode="download",
)

Finally, we'll put everything together: assign the inputs to the pipeline and set the target column (the predicted label).

In [ ]:
import uuid
from azure.ai.ml import Output

# Pipeline to construct the RAI Insights
insights_pipeline_job = rai_decision_pipeline(
    target_column_name="Diabetic",
    train_data=diabetes_train_pq,
    test_data=diabetes_test_pq,
)

# Workaround to enable the download
rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)


## Run the pipeline

When you've successfully built the pipeline, you can submit it. The following code will submit the pipeline and check the status of the pipeline. You can also view the pipeline's status in the studio. 

In [ ]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display
import time

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

When the pipeline is completed, you can review the dashboard in the studio. 